In [1]:
import pandas as pd
import numpy as np
import os
import glob
import chardet
import joblib
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import statsmodels.api as sm

In [2]:
data_2007 = pd.read_csv("2007.csv", encoding="mac_roman")

In [3]:
## Function to group time
def bin_time(crs_time):
    if crs_time == 2400:
        return 3
    return crs_time // 600


## Functin to get season based on month 
def get_season(month):
    season_dict = {(1,2,3): 'spring', (4,5,6): 'summer', (7,8,9): 'autumn', (10,11,12): 'winter'}
    for key, val in season_dict.items():
        if month in key:
            return val

In [4]:
## Data preprocessing 
data_2007['SEASON'] = data_2007['MONTH'].apply(get_season)
data_2007['CRS_DEP_BIN'] = data_2007['CRS_DEP_TIME'].apply(bin_time)   
## Definition of delay is more than 60 mins
data_2007['HAS_DEP_DELAY'] = data_2007['DEP_DELAY_GROUP'].apply(lambda x: 1 if x > 3 else 0)

## Select columns 
variables = ['DAY_OF_WEEK', 'SEASON', 'DISTANCE', 'CRS_DEP_BIN', 'PRCP_ORIGIN', 'SNOW_ORIGIN', 
                        'SNWD_ORIGIN', 'TMAX_ORIGIN', 'TMIN_ORIGIN', 'HAS_DEP_DELAY', 'DEP_DELAY']
data_2007 = data_2007[variables]
data_2007 = data_2007.dropna()

In [12]:
variables_lm = ['DEP_DELAY', 'DAY_OF_WEEK', 'SEASON', 'DISTANCE', 'CRS_DEP_BIN', 
               'PRCP_ORIGIN', 'SNOW_ORIGIN', 'SNWD_ORIGIN', 'TMAX_ORIGIN', 'TMIN_ORIGIN']
lm_data = data_2007[variables_lm]
lm_data = pd.get_dummies(lm_data, columns = ['DAY_OF_WEEK', 'CRS_DEP_BIN', 'SEASON'])

X_data_lm = lm_data.iloc[:, 1:]
y_data_lm = lm_data.iloc[: , 0]

X_lm_data = preprocessing.normalize(X_data_lm)

In [13]:
lm = LinearRegression()
lm.fit(X_data_lm, y_data_lm)

LinearRegression()

In [14]:
# add a constant to the feature matrix to fit an intercept term
X = sm.add_constant(X_data_lm)

# fit the same linear regression model using statsmodels
sm_lm = sm.OLS(y_data_lm, X).fit()

# print the summary statistics to view the p-values and R-squared value
print(sm_lm.summary())

C:\Users\ryfer\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:              DEP_DELAY   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                 1.651e+04
Date:                Sun, 09 Apr 2023   Prob (F-statistic):               0.00
Time:                        01:45:17   Log-Likelihood:            -3.4978e+07
No. Observations:             7018845   AIC:                         6.996e+07
Df Residuals:                 7018826   BIC:                         6.996e+07
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -5.879e+08   1.89e+09     -0.312